In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as pxp
import plotly.graph_objs as gph
from sklearn import metrics
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

dataframe = pd.read_csv('autoSurvey.csv')

dataframe

,driving_properties,interior,technology,comfort,reliability,handling,power,consumption,sporty,safety,gender,household
0,0,0,1,0,1,0,0,0,1,1,0,1
1,1,0,0,0,0,1,0,0,0,1,0,1
2,1,0,0,1,0,0,1,0,1,1,0,1
3,0,1,0,1,1,0,0,0,1,1,0,0
4,0,1,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
788,1,0,1,0,1,0,1,0,0,1,0,0
789,0,0,0,0,0,0,0,0,0,0,0,1
790,1,0,0,0,0,0,1,1,0,1,0,1
791,1,1,0,1,1,1,1,0,1,0,0,1


In [ ]:
covMatrix = np.cov(dataframe)
print("Covariant Martix")
print (covMatrix)

In [ ]:
import plotly.figure_factory as ff

np.random.seed(1)

fig = ff.create_dendrogram(dataframe, color_threshold = 1.5)
fig.update_layout(width=800, height=500)
fig.show()

In [ ]:
import plotly.graph_objects as go
import plotly.figure_factory as ff
from scipy.spatial.distance import pdist, squareform


# Initialize figure by creating upper dendrogram
fig = ff.create_dendrogram(dataframe, orientation='bottom')
for i in range(len(fig['data'])):
    fig['data'][i]['yaxis'] = 'y2'

# Create Side Dendrogram
dendro_side = ff.create_dendrogram(dataframe, orientation='right')
for i in range(len(dendro_side['data'])):
    dendro_side['data'][i]['xaxis'] = 'x2'

# Add Side Dendrogram Data to Figure
for data in dendro_side['data']:
    fig.add_trace(data)

# Create Heatmap
dendro_leaves = dendro_side['layout']['yaxis']['ticktext']
dendro_leaves = list(map(int, dendro_leaves))
data_dist = pdist(dataframe)
heat_data = squareform(data_dist)
heat_data = heat_data[dendro_leaves,:]
heat_data = heat_data[:,dendro_leaves]

heatmap = [
    go.Heatmap(
        x = dendro_leaves,
        y = dendro_leaves,
        z = heat_data,
        colorscale = 'Blues'
    )
]

heatmap[0]['x'] = fig['layout']['xaxis']['tickvals']
heatmap[0]['y'] = dendro_side['layout']['yaxis']['tickvals']

# Add Heatmap Data to Figure
for data in heatmap:
    fig.add_trace(data)

# Edit Layout
fig.update_layout({'width':800, 'height':800,
                         'showlegend':False, 'hovermode': 'closest',
                         })
# Edit xaxis
fig.update_layout(xaxis={'domain': [.15, 1],
                                  'mirror': False,
                                  'showgrid': False,
                                  'showline': False,
                                  'zeroline': False,
                                  'ticks':""})
# Edit xaxis2
fig.update_layout(xaxis2={'domain': [0, .15],
                                   'mirror': False,
                                   'showgrid': False,
                                   'showline': False,
                                   'zeroline': False,
                                   'showticklabels': False,
                                   'ticks':""})

# Edit yaxis
fig.update_layout(yaxis={'domain': [0, .85],
                                  'mirror': False,
                                  'showgrid': False,
                                  'showline': False,
                                  'zeroline': False,
                                  'showticklabels': False,
                                  'ticks': ""
                        })
# Edit yaxis2
fig.update_layout(yaxis2={'domain':[.825, .975],
                                   'mirror': False,
                                   'showgrid': False,
                                   'showline': False,
                                   'zeroline': False,
                                   'showticklabels': False,
                                   'ticks':""})

# Plot!
fig.show()

In [ ]:
 
# generating correlation heatmap
sns.heatmap(dataframe.corr(), annot = True)
 
# posting correlation heatmap to output console
plt.show()

In [ ]:
from matplotlib import cm
from matplotlib import colors as mcolors
from mpl_toolkits.mplot3d import Axes3D
import math

clusters = 2
''' Generating different colors in ascending order
                                of their hsv values '''
colors = list(zip(*sorted((
                    tuple(mcolors.rgb_to_hsv(
                          mcolors.to_rgba(color)[:3])), name)
                     for name, color in dict(
                            mcolors.BASE_COLORS, **mcolors.CSS4_COLORS
                                                      ).items())))[1]
  
  
# number of steps to taken generate n(clusters) colors
skips = math.floor(len(colors[5 : -5])/clusters)
cluster_colors = colors[5 : -5 : skips]

# generating correlation data
data = dataframe.corr()
data.index = range(0, len(data))
data.rename(columns = dict(zip(data.columns, data.index)), inplace = True)
data = data.astype(object)
 
''' Generating coordinates with
corresponding correlation values '''
for i in range(0, len(data)):
    for j in range(0, len(data)):
        if i != j:
            data.iloc[i, j] = (i, j, data.iloc[i, j])
        else :
            data.iloc[i, j] = (i, j, 0)
 
df_list = []
 
# flattening dataframe values
for sub_list in data.values:
    df_list.extend(sub_list)
 
# converting list of tuples into trivariate dataframe
plot_df = pd.DataFrame(df_list)
 
fig = plt.figure()
ax = Axes3D(fig)
 
# plotting 3D trisurface plot
ax.plot_trisurf(plot_df[0], plot_df[1], plot_df[2],
                    cmap = cm.jet, linewidth = 0.2)
 
plt.show()

In [ ]:
from sklearn.decomposition import PCA

x_input = dataframe.loc[:, ['driving_properties', 'interior', 'technology', 'comfort', 'reliability',
                     'handling','power', 'consumption', 'sporty', 'safety', 'gender', 'household']].values


wcss = []

for k in range(1, 12):
    k_means = KMeans(n_clusters=k, init='k-means++', random_state=0)
    k_means.fit(x_input)
    wcss.append(k_means.inertia_)
    
    
plt.figure(figsize=(15,8))

plt.plot(range(1, 12), wcss, linewidth=2, marker='8')
plt.title('Elbow method\n', fontsize=18)
plt.xlabel('K')
plt.ylabel('WCSS')
ax = plt.gca() # get axis handle
line = ax.get_lines()[0]
xydata = line.get_xydata()
plt.show()

In [ ]:
n_clusters = 2


kmeans = KMeans(n_clusters, random_state=0)

y = kmeans.fit_predict(dataframe[['driving_properties', 'interior', 'technology', 'comfort', 'reliability',
                     'handling','power', 'consumption', 'sporty', 'safety', 'gender', 'household']])

dataframe['Cluster'] = y

print(dataframe.head())

In [ ]:
# Run PCA on the data and reduce the dimensions in pca_num_components dimensions
n_clusters = 2

pca_num_components = 2
reduced_data = PCA(n_components=pca_num_components).fit_transform(dataframe)
results = pd.DataFrame(reduced_data,columns=['pca1','pca2'])

sns.scatterplot(x="pca1", y="pca2", hue=dataframe['Cluster'], data=results)
plt.title('K-means Clustering with 2 dimensions')
plt.show()

In [ ]:
from bioinfokit.visuz import cluster
from sklearn.preprocessing import StandardScaler

df = dataframe.iloc[: , :-1]
# df_st =  StandardScaler().fit_transform(df) 
pca_out = PCA(n_components=pca_num_components).fit(df)

# pca_scores = PCA().fit_transform(df_st)

loadings = pca_out.components_
pca_out.explained_variance_ratio_

cluster.biplot(show = True, cscore=reduced_data, loadings=loadings, labels=df.columns.values,
               var1=round(pca_out.explained_variance_ratio_[0]*100, 2), var2=round(pca_out.explained_variance_ratio_[1]*100, 2))
